In [1]:
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import random
from datetime import datetime

from pycaret.regression import *

pd.options.display.max_colwidth = 1000
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)

import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.float_format', '{:.2f}'.format) #evitar notacion cientifica

In [2]:
####################################################
############# Setear segun cada maquina ############
os.chdir("C:/diego_tools/labo3/dataset")
####################################################

In [3]:
seed = 456456465

In [4]:
def error_rate(y, y_pred):
    dif_abs = sum(abs(y - y_pred))
    suma_real = sum(y)
    
    return round(100*dif_abs/suma_real,2)

In [5]:
df_sellout = pd.read_csv("emp3_sellout_product_train.csv")
df_sellout.head()

,product_id,periodo,imputado,tn,cust_request_tn,cust_request_qty,plan_precios_cuidados,periodo_fecha,mes,cat1,cat2,cat3,brand,sku_size,producto_estrella,temp_media,temp_max_media,temp_min_media,IPC,promedio_mens_dolar_venta,catastrofe,tn_lag_1,tn_lag_2,tn_lag_3,tn_mas_2,tn_delta_1,tn_delta_2,tn_delta_3
0,20001,201701,0,934.77,937.73,479.00,0.00,2017-01-01,1,HC,ROPA LAVADO,Liquido,ARIEL,3000.00,1.00,25.65,30.40,20.90,1.60,15.91,False,NaN,NaN,NaN,1303.36,NaN,NaN,NaN
1,20001,201702,0,798.02,833.72,432.00,0.00,2017-02-01,2,HC,ROPA LAVADO,Liquido,ARIEL,3000.00,1.00,25.35,30.10,20.60,2.10,15.59,False,934.77,NaN,NaN,1069.96,-136.76,NaN,NaN
2,20001,201703,0,1303.36,1330.75,509.00,0.00,2017-03-01,3,HC,ROPA LAVADO,Liquido,ARIEL,3000.00,1.00,22.45,26.90,18.00,2.40,15.52,False,798.02,934.77,NaN,1502.20,505.34,368.59,NaN
3,20001,201704,0,1069.96,1132.94,279.00,0.00,2017-04-01,4,HC,ROPA LAVADO,Liquido,ARIEL,3000.00,1.00,18.90,23.50,14.30,2.70,15.36,False,1303.36,798.02,934.77,1520.07,-233.40,271.95,135.19
4,20001,201705,0,1502.20,1550.69,701.00,0.00,2017-05-01,5,HC,ROPA LAVADO,Liquido,ARIEL,3000.00,1.00,15.90,19.50,12.30,1.40,15.72,False,1069.96,1303.36,798.02,1030.67,432.24,198.84,704.19


In [6]:
df_sellout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30825 entries, 0 to 30824
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   product_id                 30825 non-null  int64  
 1   periodo                    30825 non-null  int64  
 2   imputado                   30825 non-null  int64  
 3   tn                         30825 non-null  float64
 4   cust_request_tn            30825 non-null  float64
 5   cust_request_qty           30825 non-null  float64
 6   plan_precios_cuidados      30825 non-null  float64
 7   periodo_fecha              30825 non-null  object 
 8   mes                        30825 non-null  int64  
 9   cat1                       30825 non-null  object 
 10  cat2                       30825 non-null  object 
 11  cat3                       30825 non-null  object 
 12  brand                      30825 non-null  object 
 13  sku_size                   30825 non-null  flo

In [7]:
df_train = df_sellout[df_sellout.periodo <= 201812]
df_holdout = df_sellout[df_sellout.periodo == 201902]

df_train.periodo.unique(),df_holdout.periodo.unique()

(array([201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708,
        201709, 201710, 201711, 201712, 201801, 201802, 201803, 201804,
        201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812],
       dtype=int64),
 array([201902], dtype=int64))

In [8]:
df_train_model = df_train.drop(columns=["periodo"])
df_holdout_model = df_holdout.drop(columns=["periodo"])

In [ ]:
clf = setup(
            data = df_train_model,
            test_data=df_holdout_model,
            #fold_strategy = 'timeseries',
            data_split_stratify=False,
            data_split_shuffle=False,
            session_id=seed,
            target = 'tn_mas_2')


#Metricas
previous_metric_names = get_metrics().Name

for metric_name in previous_metric_names:
    remove_metric(metric_name)

add_metric('error_rate', 'Error Rate', error_rate, greater_is_better = False)

model = compare_models  (
                        sort="error_rate"
                        )

,Description,Value
0,Session id,456456465
1,Target,tn_mas_2
2,Target type,Regression
3,Original data shape,"(30825, 27)"
4,Transformed data shape,"(30825, 68)"
5,Transformed train set shape,"(29592, 68)"
6,Transformed test set shape,"(1233, 68)"
7,Numeric features,20
8,Categorical features,5
9,Rows with missing values,12.0%


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,17:33:34
Status,. . . . . . . . . . . . . . . . . .,Compiling Final Models
Estimator,. . . . . . . . . . . . . . . . . .,K Neighbors Regressor


,Model,Error Rate,TT (Sec)
knn,K Neighbors Regressor,81.4030,0.4710
rf,Random Forest Regressor,85.3070,19.8050
dt,Decision Tree Regressor,111.5120,0.3800
huber,Huber Regressor,141.1630,0.9280
par,Passive Aggressive Regressor,208.8300,0.2860
omp,Orthogonal Matching Pursuit,456.3570,0.1640
lasso,Lasso Regression,553.0980,0.8350
en,Elastic Net,557.3390,0.8560
llar,Lasso Least Angle Regression,557.7340,0.1780
br,Bayesian Ridge,857.2550,0.2910


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

In [ ]:
plot_model(model, plot = 'feature')

In [ ]:
#interpret_model(model)

In [ ]:
prediction_holdout = predict_model(model)

prediction_holdout

In [ ]:
prediction_holdout[prediction_holdout.product_id==20001]

In [ ]:
final_best = finalize_model(model)